In [2]:
spark

In [3]:
# read file as a DataFrame
infile = '/home/cloudera/Desktop/KKBox_churn_predict/raw_data/members_v3.csv'
#infile = 'hdfs://master:8020/kkbox_churn/kkbox_data/raw_data/members_v3.csv'
df0 = spark.read.format('csv').option('header','true').option('sep',',').load(infile)

In [4]:
N = df0.count()
N

6769473

In [5]:
df0.printSchema()

root
 |-- msno: string (nullable = true)
 |-- city: string (nullable = true)
 |-- bd: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- registered_via: string (nullable = true)
 |-- registration_init_time: string (nullable = true)



In [6]:
df0.show(5)

+--------------------+----+---+------+--------------+----------------------+
|                msno|city| bd|gender|registered_via|registration_init_time|
+--------------------+----+---+------+--------------+----------------------+
|Rb9UwLQTrxzBVwCB6...|   1|  0|  null|            11|              20110911|
|+tJonkh+O1CA796Fm...|   1|  0|  null|             7|              20110914|
|cV358ssn7a0f7jZOw...|   1|  0|  null|            11|              20110915|
|9bzDeJP6sQodK73K5...|   1|  0|  null|            11|              20110915|
|WFLY3s7z4EZsieHCt...|   6| 32|female|             9|              20110915|
+--------------------+----+---+------+--------------+----------------------+
only showing top 5 rows



## bd

In [21]:
# null value in column 'bd'?
from pyspark.sql.functions import isnull
df0.where(isnull('bd')).count()

0

In [23]:
# records with 'bd' = 0
from pyspark.sql.functions import col
df0.where(col('bd') == 0).count()

4540215

* ~2/3 of the members have 'bd' = 0

## gender 

In [20]:
from pyspark.sql.functions import isnull
df0.where(isnull('gender')).count()

4429505

* ~2/3 of the members do not have 'gender' data.

### bd and gender 

In [8]:
from pyspark.sql.functions import isnull
df0.where('bd == 0').where(isnull('gender')).count()

4388613

In [21]:
from pyspark.sql.functions import expr, isnull
df0.select(expr('bd == 0').alias('bd_is_0'), 
           expr('gender IS NULL').alias('gender_is_na')) \
   .groupBy('bd_is_0', 'gender_is_na').count().show()

+-------+------------+-------+
|bd_is_0|gender_is_na|  count|
+-------+------------+-------+
|   true|       false| 151602|
|   true|        true|4388613|
|  false|       false|2188366|
|  false|        true|  40892|
+-------+------------+-------+



* members with gender data mostly have 'bd' != 0
* members without gender data mostly have 'bd' = 0

* 65% of the members have gender = NA and 'bd' = 0
* 32% of the members have gender != NA and 'bd' != 0

## city 

In [16]:
df0.groupBy('city').count().show(40)

+----+-------+
|city|  count|
+----+-------+
|   7|  11610|
|  15| 190213|
|  11|  47489|
|   3|  27282|
|   8|  45975|
|  22| 210407|
|  16|   5092|
|   5| 385069|
|  18|  38039|
|  17|  27772|
|   6| 135200|
|  19|   1199|
|   9|  47639|
|   1|4804326|
|  20|   4233|
|  10|  32482|
|   4| 246848|
|  12|  66843|
|  13| 320978|
|  14|  89940|
|  21|  30837|
+----+-------+



## registered_via

In [27]:
df0.groupBy('registered_via').count() \
   .sort(col('registered_via').cast('int')).show()

+--------------+-------+
|registered_via|  count|
+--------------+-------+
|            -1|      1|
|             1|     43|
|             2|   1452|
|             3|1643208|
|             4|2793213|
|             5|   3115|
|             6|   1213|
|             7| 805895|
|             8|   3982|
|             9|1482863|
|            10|     10|
|            11|  25047|
|            13|   5455|
|            14|    615|
|            16|    888|
|            17|   1494|
|            18|      5|
|            19|    974|
+--------------+-------+

